In [1]:
# install all these packages once
''' 
!pip install --upgrade pip
!pip install git+https://github.com/openai/whisper.git 
!pip install pytube==12.1.0
!pip install pydub==0.25.1

!pip install -q transformers==4.24.0

!pip install sentencepiece==0.1.97
!pip install git+https://github.com/csebuetnlp/normalizer

# !pip install ffmpeg==1.4
!pip install ffmpeg-python==0.2.0

#! pip install -U pip
! pip install TTS==0.8.0
! pip install bnnumerizer==0.0.2
! pip install bangla==0.0.2
! pip install gdown==4.5.1
! pip install bnunicodenormalizer==0.1.1
! pip install PyArabic==0.6.15
#! pip install pydub
! pip install Unidecode==1.3.6
#! pip install numpy
!pip install pydload==1.0.9
'''

' \n!pip install --upgrade pip\n!pip install git+https://github.com/openai/whisper.git \n!pip install pytube==12.1.0\n!pip install pydub==0.25.1\n\n!pip install -q transformers==4.24.0\n\n!pip install sentencepiece==0.1.97\n!pip install git+https://github.com/csebuetnlp/normalizer\n\n# !pip install ffmpeg==1.4\n!pip install ffmpeg-python==0.2.0\n\n#! pip install -U pip\n! pip install TTS==0.8.0\n! pip install bnnumerizer==0.0.2\n! pip install bangla==0.0.2\n! pip install gdown==4.5.1\n! pip install bnunicodenormalizer==0.1.1\n! pip install PyArabic==0.6.15\n#! pip install pydub\n! pip install Unidecode==1.3.6\n#! pip install numpy\n!pip install pydload==1.0.9\n'

In [2]:
!pwd

/home/apsisdev/mobassir/data/Multilingual-Speech-to-Speech-Translator-main


In [3]:
# ...................CONTROLS.................

make_youtube_audio_playlist = False

run_multilingual_stt = False

run_en_ar_bn_translator = False

run_multilingual_tts = True

In [4]:
import os
import soundfile as sf
import torch
import transformers
from IPython.display import Audio
print(transformers.__version__)
print(torch.__version__)

4.24.0
1.10.1+cu113


In [5]:
%%time
if(make_youtube_audio_playlist):
    !python -W ignore step1_make_youtube_audio_playlist.py


#https://towardsdatascience.com/speech-to-text-with-openais-whisper-53d5cea9005e
# import pytube
# # Reading from Youtube video link
# video = 'https://www.youtube.com/watch?v=dP--QLPUK_8'
# data = pytube.YouTube(video)
# # Converting and downloading as 'MP4' file
# audio = data.streams.get_audio_only()
# audio.download()

# !ffmpeg -i '/home/apsisdev/mobassir/data/Multilingual-Speech-to-Speech-Translator-main/Seerah_of_Prophet_Muhammed_SAW/Seerah of Prophet Muhammad 84 - Conversion of Kab ibn Zuhayr  Dr Yasir Qadhi  7th May 2014.mp4' -ac 2 -f wav '/home/apsisdev/mobassir/data/Multilingual-Speech-to-Speech-Translator-main/Seerah_of_Prophet_Muhammed_SAW/84.Seerah_of_Prophet_Muhammad_84_Conversion_of_Kab_ibn_Zuhayr_Dr_Yasir_Qadhi_7th_May_2014.wav'

CPU times: user 2 µs, sys: 1e+03 ns, total: 3 µs
Wall time: 5.01 µs


In [6]:
!ls

 arabic_tts
 audio_chunks
 bangla_seerah
 bangla_tts
 bn_translated_outputs
 e2e_mlt_stt.wav
'E2E-Multilingual Speech to Speech Translator.ipynb'
 empty.wav
 klaam
 LICENSE
 mlt_stt_outputs
 model.pth.tar
 __pycache__
 README.md
 sample.wav
 Seerah_of_Prophet_Muhammed_SAW
 step1_make_youtube_audio_playlist.py
 step2_run_multilingual_stt.py
 step3_crosslingual_translator.py
 step4_multilingual_tts.py
 tts-arabic-tacotron2
 y.q_bn_seerah
 y.q_en_seerah
 y.q_tagged_bn_seerah


In [7]:
#!ls Seerah_of_Prophet_Muhammed_SAW


In [8]:
%%time
if(run_multilingual_stt):
    from step2_run_multilingual_stt import mlt_speech_to_text_convertor
    folder_path = os.path.join(os.getcwd() , 'Seerah_of_Prophet_Muhammed_SAW') 
    
    files = sorted(os.listdir(folder_path))
    for i in range(len(files)):
        TEXTS = ''
        try:
            TEXTS = mlt_speech_to_text_convertor(path = files[i],silence_based_conversion = False)
        except:
            print("couldn't make -> ".i)

        mlt_txt = ' '.join(TEXTS)

        head, output = os.path.split(files[i])
        fname = output.split(".")[1]

        path = "mlt_stt_outputs"
        isExist = os.path.exists(path)
        if not isExist:
            os.makedirs(path)
        os.chdir(path)

        text_file = open(f"en_{fname}.txt", "w")
        n = text_file.write(mlt_txt)
        text_file.close()
        os.chdir('..')

CPU times: user 1 µs, sys: 1e+03 ns, total: 2 µs
Wall time: 3.81 µs


In [9]:
pwd

'/home/apsisdev/mobassir/data/Multilingual-Speech-to-Speech-Translator-main'

In [10]:
%%time

if(run_en_ar_bn_translator):
    from step3_crosslingual_translator import EN_AR_to_BN_AR_Translator
    folder_path = os.path.join(os.getcwd() , 'mlt_stt_outputs') 
    files = sorted(os.listdir(folder_path))
    path = "bn_translated_outputs"
    isExist = os.path.exists(path)
    if not isExist:
        os.makedirs(path)

    for i in range(len(files)):
        fpath = os.path.join(folder_path , files[i]) 
        f = open(f"{fpath}", "r")
        fread = f.read()
        try:
            bn_mlt = EN_AR_to_BN_AR_Translator(fread,tag_text = True)
        except:
            print("couldn't process -> ",i," filename -> ",files[i])
        # bn_mlt=bn_mlt.replace("</ar>",' ')
        # bn_mlt=bn_mlt.replace("<ar>",' ')

        fname = files[i]
        fname = fname.replace("en",'bn')
        fname = fname.split(".")[0]
        
        os.chdir(path)

        text_file = open(f"{fname}.txt", "w")
        n = text_file.write(bn_mlt)
        text_file.close()
        os.chdir('..')
        

CPU times: user 1 µs, sys: 0 ns, total: 1 µs
Wall time: 2.62 µs


In [ ]:

if(run_multilingual_tts):
    !git clone https://github.com/ARBML/klaam
    # make sure you have these installed https://github.com/ARBML/klaam/blob/main/requirements.txt
    from step4_multilingual_tts import BigTextToAudio
    
    folder_path = os.path.join(os.getcwd() , 'y.q_bn_seerah') 
    files = sorted(os.listdir(folder_path))
    path = "bangla_seerah"
    isExist = os.path.exists(path)
    if not isExist:
        os.makedirs(path)

    for i in range(len(files)):
        if(i>0):
            break
        fpath = os.path.join(folder_path , files[i]) 
        f = open(f"{fpath}", "r")
        fread = f.read()
        MLT_TTS=BigTextToAudio()
        fname = files[i]
#         fname = fname.replace("en",'bn')
        fname = fname.split(".")[0]
        try:
            audio=MLT_TTS(fread,resample_audio_to_out_sample_rate=False)
            sf.write(f'{fname}.wav', audio, 22050)
#             os.chdir(path)
            print(f"File {i} processing done!")
        except:
            print("couldn't process -> ",i," filename -> ",files[i])
        # bn_mlt=bn_mlt.replace("</ar>",' ')
        # bn_mlt=bn_mlt.replace("<ar>",' ')

#         os.chdir('..')
    



fatal: destination path 'klaam' already exists and is not an empty directory.
Removing weight norm...
 > Using model: vits
 > Setting up Audio Processor...
 | > sample_rate:22050
 | > resample:False
 | > num_mels:80
 | > log_func:np.log10
 | > min_level_db:0
 | > frame_shift_ms:None
 | > frame_length_ms:None
 | > ref_level_db:None
 | > fft_size:1024
 | > power:None
 | > preemphasis:0.0
 | > griffin_lim_iters:None
 | > signal_norm:None
 | > symmetric_norm:None
 | > mel_fmin:0
 | > mel_fmax:None
 | > pitch_fmin:None
 | > pitch_fmax:None
 | > spec_gain:20.0
 | > stft_pad_mode:reflect
 | > max_norm:1.0
 | > clip_norm:True
 | > do_trim_silence:False
 | > trim_db:60
 | > do_sound_norm:False
 | > do_amp_to_db_linear:True
 | > do_amp_to_db_mel:True
 | > do_rms_norm:False
 | > db_level:None
 | > stats_path:None
 | > base:10
 | > hop_length:256
 | > win_length:1024
couldn't numerize bengali.
skipped
['m', 'H', 'm', 'd']
skipped
['uu0', 'E', 'l', 'aa']
skipped
['UU0', 'S', 'H', 'b', 'h']
skipped


skipped
['l', 'l', 'h']
Raw Text Sequence: حسبنا كتاب الله
Phoneme Sequence: {H s b n aa k t aa b l l h sil}
['<BLNK>', 'ক', '<BLNK>', 'ি', '<BLNK>', 'ন', '<BLNK>', '্', '<BLNK>', 'ত', '<BLNK>', 'ু', '<BLNK>', ' ', '<BLNK>', 'আ', '<BLNK>', 'ম', '<BLNK>', 'া', '<BLNK>', 'দ', '<BLNK>', 'ে', '<BLNK>', 'র', '<BLNK>', ' ', '<BLNK>', 'ব', '<BLNK>', 'ই', '<BLNK>', '<', '<BLNK>', '>', '<BLNK>', '।', '<BLNK>']
 [!] Character '<' not found in the vocabulary. Discarding it.
['<BLNK>', 'ক', '<BLNK>', 'ি', '<BLNK>', 'ন', '<BLNK>', '্', '<BLNK>', 'ত', '<BLNK>', 'ু', '<BLNK>', ' ', '<BLNK>', 'আ', '<BLNK>', 'ম', '<BLNK>', 'া', '<BLNK>', 'দ', '<BLNK>', 'ে', '<BLNK>', 'র', '<BLNK>', ' ', '<BLNK>', 'ব', '<BLNK>', 'ই', '<BLNK>', '<', '<BLNK>', '>', '<BLNK>', '।', '<BLNK>']
 [!] Character '>' not found in the vocabulary. Discarding it.
--------------------->>>>>>>>>>>>>>>>>>>>>>>>>>>    failed ar => এটা না  end
--------------------->>>>>>>>>>>>>>>>>>>>>>>>>>>    failed ar =>  আমরা জানি না কে বলেছে এটা আর অ

In [ ]:
# text = '''<ar>بكت عيني بكت عيني بكت عيني على ذنبي وما لاقيت من كربي</ar> and do not fear from the punishment. will be the hafidh of the Quran the martyr the Shaheed and The one who gave of his money all the time for the sake of Allah these are the first three people whom Allah will call in front of him and As everybody is looking at them, they will think Allah is honoring them Allah is rewarding them Allah will give them the first share of the pie of Jannah gatherings and at these masajid. Why did you go and do this? So the hafidh will stand up with pride and he will say, Oh Allah, I did it to bring honor to your book. I did it for the izzah of the religion, for the glory of the religion. I did it for your sake, Oh Allah. And Allah will say, You're lying. And the angels will agree and say, You are lying. And the books will be barred forth and it will be clear that his intention, Allah will say, your intention was not to please me. the Quran so that people would call you Qari, Hafiz, Sheikh and they did. So get your reward from them. And then the second person, the martyr, the one who waged a legitimate expedition and he gave his life for the cause of Allah subhanahu wa ta'ala. He will be asked why did you spend your life in the way of Allah and the man will say because I wanted to bring help to the religion. I wanted to defend the religion, defend the weakless and the homeless and the poor. I did it for your sake, O Allah. And Allah will once again say, <ar>كذبت</ar>, you're lying. And the angels will testify you're lying. And Allah will say, you only did it so that people could call you, mashaAllah, Mujahid, mashaAllah, brave man, mashaAllah, you've done so much. And the people did. You got your reward from them, none from Me. And the same with the third person, and for your religion. And Allah will say you only did it so that you could be known in the community. So that people would call you generous and that is exactly what they did. And then Allah will punish these three in front of all of the creation and they will be the very first people to enter the fire of hell. Wa liyadubillah. And Abu Huraira was listening to this hadith and the Prophet tapped his knee and he said, Ya Aba Huraira, these are the first three people the Hafidh and Qari and in one rewired says the scholar and the orator may Allah azza wa jalla protect all of us and The one who was a Shaheed the martyr and the third one is the one who is generous why? Because their intentions were not correct. They had this mixing of intentions They wanted to show off their deeds and imagine brothers and sisters Imagine the fate of the person he's doing good his whole life and he thinks he's gonna meet Allah with all of this good and then he is told, you didn't do it for my sake. You did it for your own ego. You did it to inflate your own prestige on the people. Why are you wanting me to reward you when you didn't do it purely for my sake? Imagine the fate of that person and therefore brothers and sisters, we need to make sure we are not that person. We are not that person on the day of judgment who is told you wasted your whole life and all of your deeds will be thrown back at your face. <ar>اخلي ويا خجلي اذا ما قال لي ربي اما استحييته تعصيني ولا تخشى من العتبي وتخفي الذنب عن خلقي وتأبى في الهوى قربي تعود إلى رضا ربي</ar>'''

In [ ]:
# text = text.replace("<ar>"," ")
# text = text.replace("</ar>"," ")
# text

In [ ]:

# Audio(audio, rate=MLT_TTS.sample_rate)


In [ ]:
ls